# Initialization

In [3]:
import requests
from bs4 import BeautifulSoup
import re
import json
from tqdm import tqdm_notebook as tqdm
import psycopg2  
from peewee import *
from playhouse.postgres_ext import PostgresqlExtDatabase, JSONField, ArrayField
from playhouse.shortcuts import model_to_dict
import numpy as np

In [4]:
bdname='sinesearch_db'
bduser = 'postgres'
bdpassword = 'nef441'
bdhost = '51.79.69.179'
bdport = 5432

db = PostgresqlExtDatabase(bdname, user=bduser, password=bdpassword,
                           host=bdhost, port=bdport,autoconnect=True, autorollback=True)

db_new = PostgresqlExtDatabase("cs_db", user=bduser, password=bdpassword,
                           host=bdhost, port=bdport,autoconnect=True, autorollback=True)

In [5]:

from peewee import *
from playhouse.postgres_ext import PostgresqlExtDatabase, JSONField, ArrayField

class Films(Model):
    film_id=IntegerField(unique=True,index=True)
    stars = FloatField(default=0)
    level=IntegerField(default=0,null=True)
    year=IntegerField(default=2000)
    vector=ArrayField(FloatField,default=[])

    ganres=ArrayField(FloatField,default=[])
    meanage=FloatField(default=20)
    sex=FloatField(default=0.5)

    likes=IntegerField(default=10)
    dislikes=IntegerField(default=10)
    shit=IntegerField(default=0)
    marks=IntegerField(default=0)

    errors=IntegerField(default=0)
    opening=IntegerField(default=0)
    count_triler=IntegerField(default=0)

    class Meta:
        database = db_new
        db_table='Films_fp'
        
        
class FilmsFW(Model):
    film_id = IntegerField(unique=True, index=True)
    name = TextField()
    discr = TextField(null=True)
    img = TextField(null=True)
    youtube = TextField(default="")
    stars = FloatField(default=0)
    info = JSONField(default={})
    treilers = IntegerField(default=0)

    class Meta:
        database = db_new
        db_table = 'Films_fw'
        
class Users(Model):
    user_id=IntegerField(index=True,null=False)
    age=IntegerField(default=20)
    sex=IntegerField(default=0)

    ganres=ArrayField(FloatField,default=[])
    group=IntegerField(default=-1)
    mark_wight=FloatField(default=0)
    just_marked = IntegerField(default=1)

    liked=ArrayField(IntegerField,default=[])
    viewed=ArrayField(IntegerField,default=[])
    disliked=ArrayField(IntegerField,default=[])
    
    disliked_vector=ArrayField(FloatField,default=[])
    liked_vector=ArrayField(FloatField,default=[])

    predict_films=ArrayField(IntegerField,default=[])

    class Meta:
        database = db_new
        db_table='Users_fp'
        
class usualFilms(Model):
    film_id=IntegerField(unique=True,index=True)
    name = TextField()
    discr = TextField(null=True)
    img = TextField(null=True)
    youtube=TextField(default="")
    stars = FloatField(default=0)
    info=JSONField(default={})
    level=IntegerField(default=0)
    selections=ArrayField(IntegerField,default=[])
    year=IntegerField(default=2000)

    ganres=ArrayField(IntegerField,default=[])
    meanage=FloatField(default=20)
    sex=FloatField(default=0.5)

    likes=IntegerField(default=10)
    dislikes=IntegerField(default=10)
    shit=IntegerField(default=0)
    errors=IntegerField(default=0)
    opening=IntegerField(default=0)
    treilers=IntegerField(default=0)

    class Meta:
        database = db
        db_table='Films'
        
class usualUsers(Model):
    tel_id=IntegerField(index=True)
    vk_id=IntegerField()
    name = TextField()
    nicname = TextField(null=True)
    ustatus=IntegerField(default=0)
    age=IntegerField(default=20)
    sex=IntegerField(default=0)
    ganres=JSONField(default={})
    group=IntegerField(default=-1)
    info=JSONField(default={})
    last_visit=DateField(null=True)
    mark_wight=FloatField(default=0)
    just_marked = IntegerField(default=1)

    cms=IntegerField(default=0)
    cfid=IntegerField(default=0)
    cfmes=IntegerField(default=0)
    ctmes=IntegerField(default=0)
    liked=ArrayField(IntegerField,default=[])
    viewed=ArrayField(IntegerField,default=[])
    disliked=ArrayField(IntegerField,default=[])
    predict_films=ArrayField(IntegerField,default=[])
    selections=JSONField(default={})

    class Meta:
        database = db
        db_table='Users'
        
class UsersTG(Model):
    tel_id = IntegerField(index=True)
    vk_id = IntegerField()
    name = TextField()
    nicname = TextField(null=True)
    ustatus = IntegerField(default=0)
    info = JSONField(default={})
    last_visit = DateTimeField(null=True)
    just_marked = IntegerField(default=1)
    predict_films=ArrayField(IntegerField,default=[])

    cms = IntegerField(default=0)
    cfid = IntegerField(default=0)
    cfmes = IntegerField(default=0)
    ctmes = IntegerField(default=0)

    class Meta:
        database = db_new
        db_table = 'UsersTG'

In [52]:
Films.create_table()
UsersTG.create_table()
Users.create_table()
FilmsFW.create_table()

In [124]:
import datetime
UsersTG.update({UsersTG.last_visit : datetime.datetime.now()}).execute()

216

# Move TG users

In [41]:
UsersTG.drop_table()
UsersTG.create_table()

In [42]:
usualUsers.select().count()

216

In [102]:
users=[]
x=usualUsers.select().count()
for i in tqdm(range(0,x,100),total=x//100):
    res = usualUsers.select().order_by(usualUsers.id).limit(100).offset(i).execute()
    for r in res:
        users.append(model_to_dict(r))

In [100]:
users[0].keys()

dict_keys(['id', 'tel_id', 'vk_id', 'name', 'nicname', 'ustatus', 'age', 'info', 'last_visit', 'just_marked', 'cms', 'cfid', 'cfmes', 'ctmes', 'predict_films'])

In [103]:
res=[]
for u in users:
    del u["selections"]
    del u["disliked"]
    del u["viewed"]
    del u["liked"]
    del u["mark_wight"]
    del u["group"]
    del u["ganres"]
    del u["sex"]
    del u["age"]
#     del u["predict_films"]
    res.append(u)

In [104]:
res[0]

{'id': 3,
 'tel_id': 313579788,
 'vk_id': 148529786,
 'name': 'Karen Armenakyan',
 'nicname': 'Pofdj',
 'ustatus': 1,
 'info': {},
 'last_visit': datetime.date(2019, 11, 22),
 'just_marked': 381,
 'cms': 3,
 'cfid': 2670,
 'cfmes': 9534,
 'ctmes': 0,
 'predict_films': [6256, 5757]}

In [106]:
# UsersTG.drop_table()
# UsersTG.create_table()
UsersTG.insert_many(res).execute()

In [107]:
UsersTG.select().count()

216

# Move Lite Films

In [25]:
films=[]
x=usualFilms.select().count()
for i in tqdm(range(0,x,100),total=x//100):
    res = usualFilms.select().order_by(usualFilms.id).limit(100).offset(i).execute()
    for r in res:
        films.append(model_to_dict(r))

In [26]:
res=[]
for f in films:
    res.append({
        "film_id" : f["film_id"],
        "name" : f["name"],
        "discr" : f["discr"],
        "img" : f["img"],
        "youtube" : f["youtube"],
        "stars" : f["stars"],
        "info" : f["info"],
        "treilers" : f["treilers"]
    })

In [35]:
FilmsFW.create_table()

In [37]:
FilmsFW.select().count()

26392

In [36]:
# FilmsFW.insert_many(res).execute()

user_list = [FilmsFW(**r) for r in res]

# All 10 users are inserted in a single query.
FilmsFW.bulk_create(user_list,batch_size=3000)

# ================

In [57]:
with open("data/gange","r") as f:
    gange = json.load(f)

In [146]:
# Films.drop_table()
# Films.create_table()

# Users.create_table()

In [ ]:
Films.select().count()

In [110]:
import torch
import os
import json
import time
from tqdm import tqdm_notebook as tqdm
from predict import predict
model = torch.load("model_lstm_short")
model.eval()

Sequential(
  (0): Embedding(69127, 16)
  (1): LSTM(16, 128)
)

# Move films

In [80]:
all_films={}
all_films_arr=[]

for i in tqdm(range(0,26392,500)):
    res = Films.select().order_by(Films.id).limit(500).offset(i).execute()
    for f in res:
        f=model_to_dict(f)
        all_films[f["film_id"]]=f

In [ ]:
all_films={}
all_films_arr=[]

for i in tqdm(range(0,26392,500)):
    res = usualFilms.select().order_by(usualFilms.id).limit(500).offset(i).execute()
    arr=[]
    for f in res:
        f=model_to_dict(f)
        del f["id"]
        del f["name"]
        del f["img"]
        del f["youtube"]
        del f["info"]
        del f["level"]
        del f["selections"]
        f["count_triler"]=f["treilers"]
        del f["treilers"]

        ganres=[0 for i in range(26)]
        for g in f["ganres"]:
            if g>23:
                g-=1
            if g==23:
                continue # ""
            ganres[g]+=1
        f["ganres"] = ganres
        f["vector"] = model(predict(f["discr"]))[0][-1][0].tolist()
        del f["discr"]

        arr.append(f)
        all_films_arr.append(f)
        all_films[f["film_id"]]=f

#     lens=Films.insert_many(arr).on_conflict_ignore().execute()

In [81]:
with open("data/all_films_fp","w") as f:
    json.dump(all_films,f)

In [82]:
len(all_films)

26392

# Move users

In [65]:
with open("data/all_films_fp","r") as f:
    all_films=json.load(f)

In [55]:
users=[]
x=usualUsers.select().count()
for i in tqdm(range(0,x,100)):
    res = usualUsers.select().order_by(usualUsers.id).limit(100).offset(i).execute()
    for r in res:
        users.append(model_to_dict(r))

In [162]:
with open("all_films_fp","w") as f:
    json.dump(all_films,f)

In [85]:
fpusers=[]
err=[]
for u in users:
    ganres=[0 for i in range(len(gange))]
    for g in u["ganres"]:
        g2=int(g)
        if g2>23:
            g2-=1
        if g2==23:
            continue
        ganres[g2]=int(u["ganres"][g])
        
    
    
    vectors=[]
    if len(u["viewed"])==0:
        u["viewed"].append(18632)
    if len(u["liked"])==0:
        u["liked"].append(18632)
        
    u["disliked"]=[x for x in u["disliked"] if x != 0]
    if len(u["disliked"])==0:
        u["disliked"].append(1900)
        
    for f in u["viewed"]:
        try:
            vectors.append(all_films[f]["vector"])
        except:
            err.append(f)            
            
    vectors=np.array(vectors)
    vectors=vectors.mean(axis=0)
    u["viewed_vector"]=vectors
    vectors=[]
    for f in u["disliked"]:
        try:
            vectors.append(all_films[f]["vector"])
        except:
            err.append(f)
    vectors=np.array(vectors)
    vectors=vectors.mean(axis=0)
    u["disliked_vector"]=vectors
    vectors=[]
    for f in u["liked"]:
        try:
            vectors.append(all_films[f]["vector"])
        except:
            err.append(f)
    vectors=np.array(vectors)
    vectors=vectors.mean(axis=0)
    u["liked_vector"]=vectors
    u["ganres"]=ganres
    
    fpusers.append({
        "user_id":u["tel_id"],
        "disliked_vector":u["disliked_vector"].tolist(),
        "liked_vector":((u["liked_vector"]*2+u["viewed_vector"])/3).tolist(),
        "age":u["age"],
        "sex":u["sex"],
        "ganres":u["ganres"],
        "just_marked":u["just_marked"],
        "mark_wight":u["mark_wight"],
        "liked":u["liked"],
        "viewed":u["viewed"],
        "disliked":[x for x in u["disliked"] if x != 0],  
    })

In [86]:
err

[]

In [90]:
# Users.drop_table()
# Users.create_table()
UsersTG.select().count()

1

In [88]:
Users.insert_many(fpusers).on_conflict_ignore().execute()